In [1]:
# Run magic commands first

%load_ext autoreload
%autoreload 2
%cd ..

/Users/kevinvu/Root/Projects/housing-nyc


In [2]:
import urbanaccess as ua
import osmnx as ox
import pandas as pd
from geopandas import points_from_xy
from shapely.geometry import LineString

# Load osm graph

In [3]:
path = 'notebooks/data/'
G = ox.load_graphml(path + 'nycwalk.graphml')

In [82]:
gdf = ox.graph_to_gdfs(G)

In [49]:
gdf[0].head()

,y,x,street_count,highway,ref,geometry
osmid,,,,,,
30807307,40.792134,-73.962183,2,NaN,NaN,POINT (-73.96218 40.79213)
30807308,40.792024,-73.962176,2,NaN,NaN,POINT (-73.96218 40.79202)
30807309,40.791687,-73.962407,2,NaN,NaN,POINT (-73.96241 40.79169)
30807310,40.791550,-73.962498,2,NaN,NaN,POINT (-73.96250 40.79155)
30807311,40.791256,-73.962752,2,NaN,NaN,POINT (-73.96275 40.79126)


In [1]:
# add net_type = 'walk' column for osm gdf

In [9]:
gdf[1].head()

osmid    highway  oneway  reversed  length name  \
u        v          key                                                        
30807307 4276626653 0     46613689  bridleway   False     False  11.984  NaN   
         4276627019 0     46613689  bridleway   False      True  12.302  NaN   
30807308 4276626644 0    332080802    footway   False     False   5.349  NaN   
         1203806398 0    428576367    footway   False     False   5.894  NaN   
30807309 30807310   0     46613689  bridleway   False     False  17.034  NaN   

                        maxspeed lanes  ref service access width bridge  \
u        v          key                                                   
30807307 4276626653 0        NaN   NaN  NaN     NaN    NaN   NaN    NaN   
         4276627019 0        NaN   NaN  NaN     NaN    NaN   NaN    NaN   
30807308 4276626644 0        NaN   NaN  NaN     NaN    NaN   NaN    NaN   
         1203806398 0        NaN   NaN  NaN     NaN    NaN   NaN    NaN   
30807309 30807310   0        NaN   NaN  NaN     NaN    NaN   NaN    NaN   

                        junction tunnel est_width area  \
u        v          key                                  
30807307 4276626653 0        NaN    NaN       NaN  NaN   
         4276627019 0        NaN    NaN       NaN  NaN   
30807308 4276626644 0        NaN    NaN       NaN  NaN   
         1203806398 0        NaN    NaN       NaN  NaN   
30807309 30807310   0        NaN    NaN       NaN  NaN   

                                                                  geometry  
u        v          key                                                     
30807307 4276626653 0    LINESTRING (-73.96218 40.79213, -73.96224 40.7...  
         4276627019 0    LINESTRING (-73.96218 40.79213, -73.96213 40.7...  
30807308 4276626644 0    LINESTRING (-73.96218 40.79202, -73.96218 40.7...  
         1203806398 0    LINESTRING (-73.96218 40.79202, -73.96216 40.7...  
30807309 30807310   0    LINESTRING (-73.96241 40.79169, -73.96250 40.7...

In [14]:
gdf[0][['x','y']].to_numpy()

array([[-73.962183 ,  40.7921336],
       [-73.9621757,  40.7920236],
       [-73.962407 ,  40.7916869],
       ...,
       [-74.2145543,  40.5213974],
       [-74.2146434,  40.5213889],
       [-74.214722 ,  40.5222953]])

## troubleshooting saved graph file

In [8]:
print(dir(G))

['__class__', '__contains__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_adj', '_node', '_pred', '_succ', 'add_edge', 'add_edges_from', 'add_node', 'add_nodes_from', 'add_weighted_edges_from', 'adj', 'adjacency', 'adjlist_inner_dict_factory', 'adjlist_outer_dict_factory', 'clear', 'clear_edges', 'copy', 'degree', 'edge_attr_dict_factory', 'edge_key_dict_factory', 'edge_subgraph', 'edges', 'get_edge_data', 'graph', 'graph_attr_dict_factory', 'has_edge', 'has_node', 'has_predecessor', 'has_successor', 'in_degree', 'in_edges', 'is_directed', 'is_multigraph', 'name', 'nbunch_iter', 'neighbors', 'new_edge_key', 'node_attr_dict_factory', 'node_dict_factory', 'nodes', 'n

In [14]:
G.graph = {'created_date': '2022-11-27 18:28:07',
 'created_with': 'OSMnx 1.2.2',
 'crs': 'epsg:4326'}

In [9]:
from geopandas import read_file
nodes = read_file(path + 'nycwalk.gpkg', layer='nodes')
edges = read_file(path+ 'nycwalk.gpkg', layer='edges')

In [28]:
g = list(G.nodes(data=True))
display(g[0])

(30807307, {'y': 40.7921336, 'x': -73.962183, 'street_count': 2})

# Load subway graph

In [83]:
subway_nodes = pd.read_csv(path + 'subway_nodes.csv', index_col='Id')
subway_edges = pd.read_csv(path + 'subway_edges.csv')

In [64]:
subway_nodes.head()

,x,y,unique_agency_id,route_type,stop_id,stop_name,parent_station,stop_code,zone_id,location_type,net_type
Id,,,,,,,,,,,
N05N_Q_mta_new_york_city_transit,-73.990414,40.620671,mta_new_york_city_transit,1.0,N05N,18 Av,N05,NaN,NaN,0,transit
Q01N_Q_mta_new_york_city_transit,-74.000460,40.718383,mta_new_york_city_transit,1.0,Q01N,Canal St,Q01,NaN,NaN,0,transit
249S_2_mta_new_york_city_transit,-73.942161,40.669399,mta_new_york_city_transit,1.0,249S,Kingston Av,249,NaN,NaN,0,transit
D13S_B_mta_new_york_city_transit,-73.944216,40.824783,mta_new_york_city_transit,1.0,D13S,145 St,D13,NaN,NaN,0,transit
635S_4_mta_new_york_city_transit,-73.989951,40.734673,mta_new_york_city_transit,1.0,635S,14 St-Union Sq,635,NaN,NaN,0,transit


In [19]:
subway_edges.head()

,Source,Target,unique_agency_id,route_type,net_type,total_trips,weight
0,101S_1_mta_new_york_city_transit,103S_1_mta_new_york_city_transit,mta_new_york_city_transit,1,transit,175.0,1.528571
1,103N_1_mta_new_york_city_transit,101N_1_mta_new_york_city_transit,mta_new_york_city_transit,1,transit,177.0,1.500000
2,103S_1_mta_new_york_city_transit,104S_1_mta_new_york_city_transit,mta_new_york_city_transit,1,transit,191.0,1.500000
3,104N_1_mta_new_york_city_transit,103N_1_mta_new_york_city_transit,mta_new_york_city_transit,1,transit,177.0,2.652542
4,104S_1_mta_new_york_city_transit,106S_1_mta_new_york_city_transit,mta_new_york_city_transit,1,transit,191.0,1.500000


In [32]:
subway_nodes[['x','y']]

,x,y
0,-73.990414,40.620671
1,-74.000460,40.718383
2,-73.942161,40.669399
3,-73.944216,40.824783
4,-73.989951,40.734673
...,...,...
1900,-74.004131,40.713065
1901,-73.844521,40.721691
1902,-73.801109,40.702147
1903,-73.889601,40.711396


## build geometry

In [36]:
len(dt)

1905

In [84]:
dt = subway_nodes[['x','y']]
geom_n = points_from_xy(dt['x'], dt['y'], crs='epsg:4326')
subway_nodes['geometry'] = geom_n

In [85]:
dt = subway_edges[['Source', 'Target']]
geom_e = []
for i in range(len(dt)):
    geom_e.append(LineString([subway_nodes['geometry'].loc[dt['Source'].iloc[i]], 
                             subway_nodes['geometry'].loc[dt['Target'].iloc[i]]]).wkt)
subway_edges['geometry'] = geom_e

# Develop integration function

Use as many original urbanaccess functions as possible. This section is basically urbanaccess.network.integrate_network() with headways=False arg, and modified dataframe names.

## build edges connecting subway and walk

note that "transit" means subway here, and "osm" means walk

In [86]:
connector_edges = ua.network._connector_edges(gdf[0], subway_nodes)

Connector edges between the OSM and transit network nodes successfully completed. Took 1.43 seconds


In [87]:
connector_edges.head()

,from,to,weight,net_type
0,N05N_Q_mta_new_york_city_transit,5241489834,1.500812,transit to osm
1,5241489834,N05N_Q_mta_new_york_city_transit,1.500812,osm to transit
2,Q01N_Q_mta_new_york_city_transit,8307641786,0.008863,transit to osm
3,8307641786,Q01N_Q_mta_new_york_city_transit,0.008863,osm to transit
4,249S_2_mta_new_york_city_transit,42467466,0.040224,transit to osm


## rename columns for merging

In [88]:
gdf[0].reset_index(inplace=True, drop=False)
gdf[0].rename(columns={'osmid': 'id'}, inplace=True)
gdf[1].reset_index(inplace=True, drop=False)
gdf[1].rename(columns={'osmid': 'id', 'u': 'from', 'v': 'to', 'length': 'weight'}, inplace=True)
# gdf[1].index.set_names(['from', 'to', 'key'], inplace=True)

In [89]:
display(gdf[0].head())
gdf[1].head()

,id,y,x,street_count,highway,ref,geometry
0,30807307,40.792134,-73.962183,2,NaN,NaN,POINT (-73.96218 40.79213)
1,30807308,40.792024,-73.962176,2,NaN,NaN,POINT (-73.96218 40.79202)
2,30807309,40.791687,-73.962407,2,NaN,NaN,POINT (-73.96241 40.79169)
3,30807310,40.791550,-73.962498,2,NaN,NaN,POINT (-73.96250 40.79155)
4,30807311,40.791256,-73.962752,2,NaN,NaN,POINT (-73.96275 40.79126)


,from,to,key,id,highway,oneway,reversed,weight,name,maxspeed,...,ref,service,access,width,bridge,junction,tunnel,est_width,area,geometry
0,30807307,4276626653,0,46613689,bridleway,False,False,11.984,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-73.96218 40.79213, -73.96224 40.7..."
1,30807307,4276627019,0,46613689,bridleway,False,True,12.302,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-73.96218 40.79213, -73.96213 40.7..."
2,30807308,4276626644,0,332080802,footway,False,False,5.349,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-73.96218 40.79202, -73.96218 40.7..."
3,30807308,1203806398,0,428576367,footway,False,False,5.894,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-73.96218 40.79202, -73.96216 40.7..."
4,30807309,30807310,0,46613689,bridleway,False,False,17.034,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-73.96241 40.79169, -73.96250 40.7..."


In [90]:
subway_nodes.reset_index(inplace=True, drop=False)
subway_nodes.rename(columns={'Id': 'id'}, inplace=True)
subway_edges.rename(columns={'Source': 'from', 'Target': 'to'}, inplace=True)

In [91]:
display(subway_nodes.head())
subway_edges.head()

,id,x,y,unique_agency_id,route_type,stop_id,stop_name,parent_station,stop_code,zone_id,location_type,net_type,geometry,nearest_osm_node
0,N05N_Q_mta_new_york_city_transit,-73.990414,40.620671,mta_new_york_city_transit,1.0,N05N,18 Av,N05,NaN,NaN,0,transit,POINT (-73.99041 40.62067),5241489834
1,Q01N_Q_mta_new_york_city_transit,-74.000460,40.718383,mta_new_york_city_transit,1.0,Q01N,Canal St,Q01,NaN,NaN,0,transit,POINT (-74.00046 40.71838),8307641786
2,249S_2_mta_new_york_city_transit,-73.942161,40.669399,mta_new_york_city_transit,1.0,249S,Kingston Av,249,NaN,NaN,0,transit,POINT (-73.94216 40.66940),42467466
3,D13S_B_mta_new_york_city_transit,-73.944216,40.824783,mta_new_york_city_transit,1.0,D13S,145 St,D13,NaN,NaN,0,transit,POINT (-73.94422 40.82478),4205440426
4,635S_4_mta_new_york_city_transit,-73.989951,40.734673,mta_new_york_city_transit,1.0,635S,14 St-Union Sq,635,NaN,NaN,0,transit,POINT (-73.98995 40.73467),10174519086


,from,to,unique_agency_id,route_type,net_type,total_trips,weight,geometry
0,101S_1_mta_new_york_city_transit,103S_1_mta_new_york_city_transit,mta_new_york_city_transit,1,transit,175.0,1.528571,"LINESTRING (-73.898583 40.889248, -73.90087 40..."
1,103N_1_mta_new_york_city_transit,101N_1_mta_new_york_city_transit,mta_new_york_city_transit,1,transit,177.0,1.500000,"LINESTRING (-73.90087 40.884667, -73.898583 40..."
2,103S_1_mta_new_york_city_transit,104S_1_mta_new_york_city_transit,mta_new_york_city_transit,1,transit,191.0,1.500000,"LINESTRING (-73.90087 40.884667, -73.904834 40..."
3,104N_1_mta_new_york_city_transit,103N_1_mta_new_york_city_transit,mta_new_york_city_transit,1,transit,177.0,2.652542,"LINESTRING (-73.904834 40.878856, -73.90087 40..."
4,104S_1_mta_new_york_city_transit,106S_1_mta_new_york_city_transit,mta_new_york_city_transit,1,transit,191.0,1.500000,"LINESTRING (-73.904834 40.878856, -73.909831 4..."


### build geometry of connector edges and concat network nodes

In [92]:
net_nodes = pd.concat([subway_nodes, gdf[0]], axis=0, ignore_index=False)

In [93]:
display(len(net_nodes))
print(len(subway_nodes) + len(gdf[0]))

617997

617997


In [94]:
net_nodes.head()

,id,x,y,unique_agency_id,route_type,stop_id,stop_name,parent_station,stop_code,zone_id,location_type,net_type,geometry,nearest_osm_node,street_count,highway,ref
0,N05N_Q_mta_new_york_city_transit,-73.990414,40.620671,mta_new_york_city_transit,1.0,N05N,18 Av,N05,NaN,NaN,0.0,transit,POINT (-73.99041 40.62067),5.241490e+09,NaN,NaN,NaN
1,Q01N_Q_mta_new_york_city_transit,-74.000460,40.718383,mta_new_york_city_transit,1.0,Q01N,Canal St,Q01,NaN,NaN,0.0,transit,POINT (-74.00046 40.71838),8.307642e+09,NaN,NaN,NaN
2,249S_2_mta_new_york_city_transit,-73.942161,40.669399,mta_new_york_city_transit,1.0,249S,Kingston Av,249,NaN,NaN,0.0,transit,POINT (-73.94216 40.66940),4.246747e+07,NaN,NaN,NaN
3,D13S_B_mta_new_york_city_transit,-73.944216,40.824783,mta_new_york_city_transit,1.0,D13S,145 St,D13,NaN,NaN,0.0,transit,POINT (-73.94422 40.82478),4.205440e+09,NaN,NaN,NaN
4,635S_4_mta_new_york_city_transit,-73.989951,40.734673,mta_new_york_city_transit,1.0,635S,14 St-Union Sq,635,NaN,NaN,0.0,transit,POINT (-73.98995 40.73467),1.017452e+10,NaN,NaN,NaN


In [102]:
geom_conn = []
dt = net_nodes.set_index('id')

for i in list(connector_edges[['from', 'to']].itertuples(index=False, name=None)):
    p1 = dt['geometry'].loc[i[0]]
    p2 = dt['geometry'].loc[i[1]]
    geom_conn.append(LineString([p1, p2]).wkt)

connector_edges['geometry'] = geom_conn

## concat all network edges

In [103]:
net_edges = pd.concat([subway_edges, gdf[1], connector_edges], axis=0)
net_edges, net_nodes = ua.network._format_pandana_edges_nodes(edge_df=net_edges, node_df=net_nodes)

Edge and node tables formatted for Pandana with integer node ids: id_int, to_int, and from_int. Took 77.15 seconds


In [104]:
display(net_nodes.head())
net_edges.head()

,id,x,y,unique_agency_id,route_type,stop_id,stop_name,parent_station,stop_code,zone_id,location_type,net_type,geometry,street_count,highway,ref
id_int,,,,,,,,,,,,,,,,
1,N05N_Q_mta_new_york_city_transit,-73.990414,40.620671,mta_new_york_city_transit,1.0,N05N,18 Av,N05,NaN,NaN,0.0,transit,POINT (-73.99041 40.62067),NaN,NaN,NaN
2,Q01N_Q_mta_new_york_city_transit,-74.000460,40.718383,mta_new_york_city_transit,1.0,Q01N,Canal St,Q01,NaN,NaN,0.0,transit,POINT (-74.00046 40.71838),NaN,NaN,NaN
3,249S_2_mta_new_york_city_transit,-73.942161,40.669399,mta_new_york_city_transit,1.0,249S,Kingston Av,249,NaN,NaN,0.0,transit,POINT (-73.94216 40.66940),NaN,NaN,NaN
4,D13S_B_mta_new_york_city_transit,-73.944216,40.824783,mta_new_york_city_transit,1.0,D13S,145 St,D13,NaN,NaN,0.0,transit,POINT (-73.94422 40.82478),NaN,NaN,NaN
5,635S_4_mta_new_york_city_transit,-73.989951,40.734673,mta_new_york_city_transit,1.0,635S,14 St-Union Sq,635,NaN,NaN,0.0,transit,POINT (-73.98995 40.73467),NaN,NaN,NaN


,from,to,unique_agency_id,route_type,net_type,total_trips,weight,geometry,key,edge_id,...,service,access,width,bridge,junction,tunnel,est_width,area,from_int,to_int
0,101S_1_mta_new_york_city_transit,103S_1_mta_new_york_city_transit,mta_new_york_city_transit,1,transit,175.0,1.528571,"LINESTRING (-73.898583 40.889248, -73.90087 40...",NaN,NaN,...,nan,nan,nan,nan,nan,nan,nan,nan,1307,1524
1,103N_1_mta_new_york_city_transit,101N_1_mta_new_york_city_transit,mta_new_york_city_transit,1,transit,177.0,1.500000,"LINESTRING (-73.90087 40.884667, -73.898583 40...",NaN,NaN,...,nan,nan,nan,nan,nan,nan,nan,nan,902,1882
2,103S_1_mta_new_york_city_transit,104S_1_mta_new_york_city_transit,mta_new_york_city_transit,1,transit,191.0,1.500000,"LINESTRING (-73.90087 40.884667, -73.904834 40...",NaN,NaN,...,nan,nan,nan,nan,nan,nan,nan,nan,1524,221
3,104N_1_mta_new_york_city_transit,103N_1_mta_new_york_city_transit,mta_new_york_city_transit,1,transit,177.0,2.652542,"LINESTRING (-73.904834 40.878856, -73.90087 40...",NaN,NaN,...,nan,nan,nan,nan,nan,nan,nan,nan,141,902
4,104S_1_mta_new_york_city_transit,106S_1_mta_new_york_city_transit,mta_new_york_city_transit,1,transit,191.0,1.500000,"LINESTRING (-73.904834 40.878856, -73.909831 4...",NaN,NaN,...,nan,nan,nan,nan,nan,nan,nan,nan,221,861


In [105]:
all(net_edges['geometry'])

True

## save integrated network

In [106]:
type(gdf[0])

geopandas.geodataframe.GeoDataFrame

In [107]:
type(net_nodes)

pandas.core.frame.DataFrame

In [108]:
type(net_nodes['geometry'].loc[1906])

shapely.geometry.point.Point

In [109]:
net_nodes.loc[net_nodes['id'] == '30807307']

,id,x,y,unique_agency_id,route_type,stop_id,stop_name,parent_station,stop_code,zone_id,location_type,net_type,geometry,street_count,highway,ref
id_int,,,,,,,,,,,,,,,,
1906,30807307,-73.962183,40.792134,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-73.96218 40.79213),2.0,NaN,NaN


In [110]:
len(net_nodes.loc[net_nodes['geometry'] != None])

617997

In [111]:
len(net_nodes)

617997

In [112]:
net_nodes.to_csv(path + 'net_nodes.csv')
net_edges.to_csv(path + 'net_edges.csv')

In [113]:
for i in range(len(net_nodes.columns)):
    print("'" + net_nodes.columns[i] + "': " + "'" + str(net_nodes.dtypes[i]) + "',")

'id': 'object',
'x': 'float64',
'y': 'float64',
'unique_agency_id': 'object',
'route_type': 'float64',
'stop_id': 'object',
'stop_name': 'object',
'parent_station': 'object',
'stop_code': 'float64',
'zone_id': 'float64',
'location_type': 'float64',
'net_type': 'object',
'geometry': 'geometry',
'street_count': 'float64',
'highway': 'object',
'ref': 'object',


In [114]:
for i in range(len(net_edges.columns)):
    print("'" + net_edges.columns[i] + "': " + "'" + str(net_edges.dtypes[i]) + "',")

'from': 'object',
'to': 'object',
'unique_agency_id': 'object',
'route_type': 'object',
'net_type': 'object',
'total_trips': 'float64',
'weight': 'float64',
'geometry': 'object',
'key': 'float64',
'edge_id': 'float64',
'highway': 'object',
'oneway': 'object',
'reversed': 'object',
'name': 'object',
'maxspeed': 'object',
'lanes': 'object',
'ref': 'object',
'service': 'object',
'access': 'object',
'width': 'object',
'bridge': 'object',
'junction': 'object',
'tunnel': 'object',
'est_width': 'object',
'area': 'object',
'from_int': 'int64',
'to_int': 'int64',
